# Install Necessary Libraries

In [1]:
import sys

!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras

# Import Required Materials for Genre Identification

In [13]:
# feature extraction and data preprocessing
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

# Acquire and Prepare Dataset

In [14]:
data = pd.read_csv('Data/features_30_sec.csv')
data = data.drop(['filename', 'length', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo'],axis=1)
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [15]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

# Setup Training vs Testing Data


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X_train[10]

array([ 0.10902627,  1.98186297,  1.17064536,  2.70723942,  1.53034596,
        4.10373031,  1.56358825,  0.85678142,  1.48608373,  2.17604728,
        0.43588817,  2.97865236,  0.3422116 ,  0.89427392, -1.62937014,
        1.87602883,  1.77904405, -0.05732415, -1.0724823 ,  1.80241285,
        0.05452017,  0.36662529, -0.84368444,  1.28921598,  0.28809568,
        0.91124036, -1.59935865,  1.82061136,  0.58277088,  1.36826341,
       -0.99470941,  1.41807375,  0.78349371,  0.83379966, -0.58527428,
        2.09777043,  1.08730133,  1.33428423, -0.56586955,  1.24240225,
        1.14924467,  0.15038996, -0.07061864,  0.13793283,  0.50179011,
        0.45450098,  0.55574824,  0.17525067,  1.44577531,  0.79796768,
        1.26389643,  0.33293702])

# Setup Network Arhitecture and Define Network Loss and Optimization

In [17]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(2048, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the network

In [18]:
history = model.fit(X_train,
                    y_train,
                    epochs=1000,
                    batch_size=800)

Epoch 1/1000
2/2 [==============================] - 0s 24ms/step - loss: 2.2893 - accuracy: 0.1305
Epoch 2/1000
2/2 [==============================] - 0s 32ms/step - loss: 1.9767 - accuracy: 0.3758
Epoch 3/1000
2/2 [==============================] - 0s 35ms/step - loss: 1.5609 - accuracy: 0.4653
Epoch 4/1000
2/2 [==============================] - 0s 24ms/step - loss: 1.2610 - accuracy: 0.5411
Epoch 5/1000
2/2 [==============================] - 0s 27ms/step - loss: 1.2610 - accuracy: 0.5474
Epoch 6/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.1595 - accuracy: 0.5926
Epoch 7/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.0496 - accuracy: 0.6200
Epoch 8/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.8896 - accuracy: 0.6821
Epoch 9/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.7930 - accuracy: 0.7232
Epoch 10/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.7682 - accuracy: 0.7021
Epoch 11/

2/2 [==============================] - 0s 21ms/step - loss: 0.0105 - accuracy: 0.9979
Epoch 84/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.0039 - accuracy: 0.9989
Epoch 85/1000
2/2 [==============================] - 0s 8ms/step - loss: 0.0153 - accuracy: 0.9937
Epoch 86/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0042 - accuracy: 0.9989
Epoch 87/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0114 - accuracy: 0.9989
Epoch 88/1000
2/2 [==============================] - 0s 20ms/step - loss: 0.0039 - accuracy: 0.9989
Epoch 89/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.0077 - accuracy: 0.9989
Epoch 90/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0030 - accuracy: 0.9989
Epoch 91/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.0036 - accuracy: 0.9989
Epoch 92/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 93/1000
2

2/2 [==============================] - 0s 19ms/step - loss: 0.0031 - accuracy: 0.9989
Epoch 165/1000
2/2 [==============================] - 0s 21ms/step - loss: 0.0024 - accuracy: 0.9989
Epoch 166/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 167/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 168/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.0019 - accuracy: 0.9989
Epoch 169/1000
2/2 [==============================] - 0s 22ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 170/1000
2/2 [==============================] - 0s 21ms/step - loss: 0.0019 - accuracy: 0.9989
Epoch 171/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.0022 - accuracy: 0.9989
Epoch 172/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.0024 - accuracy: 0.9989
Epoch 173/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.0025 - accuracy: 0.9989
Epoch

2/2 [==============================] - 0s 20ms/step - loss: 0.0047 - accuracy: 0.9989
Epoch 246/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0035 - accuracy: 0.9989
Epoch 247/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0043 - accuracy: 0.9989
Epoch 248/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0024 - accuracy: 0.9989
Epoch 249/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0072 - accuracy: 0.9989
Epoch 250/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0102 - accuracy: 0.9989
Epoch 251/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0064 - accuracy: 0.9989
Epoch 252/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0034 - accuracy: 0.9989
Epoch 253/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0034 - accuracy: 0.9989
Epoch 254/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0031 - accuracy: 0.9989
Epoch

2/2 [==============================] - 0s 15ms/step - loss: 0.0027 - accuracy: 0.9989
Epoch 327/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0019 - accuracy: 0.9989
Epoch 328/1000
2/2 [==============================] - 0s 21ms/step - loss: 0.0017 - accuracy: 0.9989
Epoch 329/1000
2/2 [==============================] - 0s 23ms/step - loss: 0.0029 - accuracy: 0.9989
Epoch 330/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.0038 - accuracy: 0.9989
Epoch 331/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.0032 - accuracy: 0.9989
Epoch 332/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.0021 - accuracy: 0.9989
Epoch 333/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0017 - accuracy: 0.9989
Epoch 334/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0024 - accuracy: 0.9989
Epoch 335/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0023 - accuracy: 0.9989
Epoch

...

2/2 [==============================] - 0s 28ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 651/1000
2/2 [==============================] - 0s 23ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 652/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 653/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 654/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 655/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.0020 - accuracy: 0.9989
Epoch 656/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0022 - accuracy: 0.9989
Epoch 657/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 658/1000
2/2 [==============================] - 0s 22ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 659/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0018 - accuracy: 0.9979
Epoch

2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 732/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 733/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 734/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 735/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 736/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 737/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 738/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 739/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 740/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch

2/2 [==============================] - 0s 18ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 813/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 814/1000
2/2 [==============================] - 0s 24ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 815/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 816/1000
2/2 [==============================] - 0s 25ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 817/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0017 - accuracy: 0.9989
Epoch 818/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0017 - accuracy: 0.9989
Epoch 819/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 820/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch 821/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0018 - accuracy: 0.9989
Epoch

2/2 [==============================] - 0s 15ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 894/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 895/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 896/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 897/1000
2/2 [==============================] - 0s 25ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 898/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 899/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 900/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 901/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 902/1000
2/2 [==============================] - 0s 21ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch

2/2 [==============================] - 0s 16ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 975/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 976/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 977/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 978/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 979/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 980/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 981/1000
2/2 [==============================] - 0s 23ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 982/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 983/1000
2/2 [==============================] - 0s 18ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch

# Evaluate the model with our testing data

In [19]:
test_loss, test_acc = model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 11ms/step - loss: 0.9794 - accuracy: 0.8600


In [20]:
print('test_acc: ',test_acc)

test_acc:  0.8600000143051147


# Prepare Our Own CSV File Made Up Of 5 Songs

In [21]:
header = 'filename chroma_stft_mean chroma_stft_var rmse_mean rmse_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var'
for i in range(1, 21):
    header += f' mfcc{i}_mean'
    header += f' mfcc{i}_var'
header += ' label'
header = header.split()

file = open('Data/ourData.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

Importing: Erik Satie - Gnossienne No. 1

In [22]:
#
# Classical Song: Erik Satie - Gnossienne No. 1 #
#

songname = 'Data/My Songs/Erik Satie Gnossienne No 1.wav'
filename = 'Classical'
        
y, sr = librosa.load(songname, mono=True, duration=30)

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 20)

to_append = f'{filename} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
    to_append += f' {np.var(e)}'

file = open('Data/ourData.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

Importing: Slipknot - Psychosocial

In [23]:
#    
# Metal Song: Slipknot - Psychosocial #
#

songname = 'Data/My Songs/Slipknot Psychosocial.wav'
filename = 'Metal'
        
y, sr = librosa.load(songname, mono=True, duration=30)

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 20)

to_append = f'{filename} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
    to_append += f' {np.var(e)}'

file = open('Data/ourData.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

Importing: Kanye West - Homecoming

In [24]:
#    
# Hiphop Song: Kanye West - Homecoming #
#

songname = 'Data/My Songs/Kanye West Homecoming.wav'
filename = 'Hiphop'
        
y, sr = librosa.load(songname, mono=True, duration=30)

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 20)

to_append = f'{filename} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
    to_append += f' {np.var(e)}'

file = open('Data/ourData.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

Importing: Earth, Wind & Fire - Boogie Wonderland

In [25]:
#    
# Disco Song: Earth, Wind & Fire - Boogie Wonderland #
#

songname = 'Data/My Songs/Earth, Wind & Fire Boogie Wonderland.wav'
filename = 'Disco'
        
y, sr = librosa.load(songname, mono=True, duration=30)

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 20)

to_append = f'{filename} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
    to_append += f' {np.var(e)}'

file = open('Data/ourData.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

Importing: Luke Bryan - Huntin’, Fishin’, And Lovin’ Every Day

In [26]:
#    
# Country: Luke Bryan - Huntin’, Fishin’, And Lovin’ Every Day #
#

songname = 'Data/My Songs/Luke Bryan Huntin Fishin And Lovin Every Day.wav'
filename = 'Country'
        
y, sr = librosa.load(songname, mono=True, duration=30)

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 20)

to_append = f'{filename} {np.mean(chroma_stft)} {np.var(chroma_stft)} {np.mean(rmse)} {np.var(rmse)} {np.mean(spec_cent)} {np.var(spec_cent)} {np.mean(spec_bw)} {np.var(spec_bw)} {np.mean(rolloff)} {np.var(rolloff)} {np.mean(zcr)} {np.var(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
    to_append += f' {np.var(e)}'

file = open('Data/ourData.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

In [27]:
mySongs = pd.read_csv('Data/ourData.csv')
mySongs = mySongs.drop(['filename'],axis=1)

playlist = scaler.fit_transform(np.array(mySongs.iloc[:, :-1], dtype = float))

mySongs.head()

,chroma_stft_mean,chroma_stft_var,rmse_mean,rmse_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.211131,0.081814,0.012802,0.000100,623.063834,6.085770e+04,867.695632,85801.557283,869.278077,2.471230e+05,...,85.666458,-2.370523,66.895195,1.009696,89.807236,-0.809346,54.214386,-2.533461,54.951843,NaN
1,0.565246,0.065401,0.143160,0.004589,2738.083006,5.427388e+05,2403.026068,323606.627204,5472.708574,2.092138e+06,...,23.453413,-6.418691,27.012348,-0.034738,19.865368,-4.952381,19.291584,1.456171,17.024811,NaN
2,0.313548,0.093817,0.062155,0.000536,2126.347715,1.030973e+06,2366.533836,394820.043855,4614.947094,4.488513e+06,...,50.455727,-5.963078,57.258476,2.658616,59.878265,-3.237016,59.289085,-0.736721,52.538868,NaN
3,0.454574,0.078355,0.100147,0.003560,2361.768447,3.195751e+05,2491.737516,141406.663481,5135.743945,1.785178e+06,...,32.650120,-3.330728,28.787672,0.587025,37.896782,-4.136004,42.707653,1.450542,55.696690,NaN
4,0.326431,0.091957,0.046247,0.000440,1373.177032,3.625729e+05,1880.889347,291742.834824,2992.531905,2.299134e+06,...,42.838161,-4.204693,49.972633,2.667310,34.793884,-6.624167,59.084354,-2.653917,49.193020,NaN


# Model Inference 

In [28]:
print(X_train.shape)
print(playlist.shape)

predictions = model.predict(playlist)

print(predictions.shape)

(950, 52)
(5, 52)
(5, 10)


In [29]:
print("A. [Erik Satie - Gnossienne No. 1], is a Classical Song. The Neural Network has decided it's top two genre's are: \n\t1. ~", genre_list[np.argmax(predictions[0]) * 100], "~")
predictionList = list(predictions[0])
print("\t2. ~",genre_list[([predictionList.index(i) for i in sorted(predictionList, reverse=True)][:2])[1]* 100], "~\n")

print("B. [Slipknot - Psychosocial], is a Metal Song. The Neural Network has decided it's top two genre's are: \n\t1. ~", genre_list[np.argmax(predictions[1]) * 100], "~")
predictionList = list(predictions[1])
print("\t2. ~",genre_list[([predictionList.index(i) for i in sorted(predictionList, reverse=True)][:2])[1]* 100], "~\n")

print("C. [Kanye West - Homecoming], is a Hiphop Song. The Neural Network has decided it's top two genre's are: \n\t1. ~", genre_list[np.argmax(predictions[2]) * 100], "~")
predictionList = list(predictions[2])
print("\t2. ~",genre_list[([predictionList.index(i) for i in sorted(predictionList, reverse=True)][:2])[1]* 100], "~\n")

print("D. [Earth, Wind & Fire - Boogie Wonderland], is a Disco Song. The Neural Network has decided it's top two genre's are: \n\t1. ~", genre_list[np.argmax(predictions[3]) * 100], "~")
predictionList = list(predictions[3])
print("\t2. ~",genre_list[([predictionList.index(i) for i in sorted(predictionList, reverse=True)][:2])[1]* 100], "~\n")

print("E. [Luke Bryan - Huntin’, Fishin’, And Lovin’ Every Day], is a Country Song. The Neural Network has decided it's top two genre's are: \n\t1. ~", genre_list[np.argmax(predictions[4]) * 100], "~")
predictionList = list(predictions[4])
print("\t2. ~",genre_list[([predictionList.index(i) for i in sorted(predictionList, reverse=True)][:2])[1]* 100], "~\n")

A. [Erik Satie - Gnossienne No. 1], is a Classical Song. The Neural Network has decided it's top two genre's are: 
	1. ~ jazz ~
	2. ~ classical ~

B. [Slipknot - Psychosocial], is a Metal Song. The Neural Network has decided it's top two genre's are: 
	1. ~ metal ~
	2. ~ rock ~

C. [Kanye West - Homecoming], is a Hiphop Song. The Neural Network has decided it's top two genre's are: 
	1. ~ reggae ~
	2. ~ country ~

D. [Earth, Wind & Fire - Boogie Wonderland], is a Disco Song. The Neural Network has decided it's top two genre's are: 
	1. ~ disco ~
	2. ~ pop ~

E. [Luke Bryan - Huntin’, Fishin’, And Lovin’ Every Day], is a Country Song. The Neural Network has decided it's top two genre's are: 
	1. ~ country ~
	2. ~ reggae ~

